In [2]:
import os
import plotly.express as px
import torch as t
from torch import Tensor
import numpy as np
import einops
from jaxtyping import Int, Float
from typing import List, Optional, Tuple
from tqdm import tqdm
from IPython.display import display
import pickle
import torch
from dataclasses import dataclass
from typing import Dict, Optional
from jaxtyping import Float, Int
from torch import Tensor
from torch.utils.data import DataLoader
from transformer_lens.hook_points import HookPoint
from transformer_lens import utils, HookedTransformer, ActivationCache
import circuitsvis as cv
from transformer_lens.utils import tokenize_and_concatenate
from torch.utils.data import DataLoader, Dataset
import datasets

num_epochs=100
batch_size=16
weight_decay=1e-4
device="cuda:0"
lr = 1e-4
betas = (0.9, 0.99)
max_grad_norm = 1.0
wd = 0.1
print_every = 20

rhyme_model: HookedTransformer = HookedTransformer.from_pretrained("gpt2-small")

/cta/users/basan/.conda/envs/mech/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loaded pretrained model gpt2-small into HookedTransformer


In [2]:
class ShakespeareDataset(Dataset):
    def __init__(self, tokenizer, data_path="Eminem_Lyrics.txt", block_size=32, cl=1024):
        with open (data_path, 'r') as f:
            self.data = f.read()
        self.block_size = block_size
        self.tokenizer = tokenizer
        self.data = self.tokenizer(self.data)["input_ids"]
        self.dataset_size = torch.arange(0, len(self.data)-1, cl)
        self.x = [self.data[i:i+self.block_size] for i in self.dataset_size]
        self.y = [self.data[i+1:i+self.block_size+1] for i in self.dataset_size]

    def tokenize(self):
        chars = sorted(list(set(self.data)))
        self.vocab_size = len(chars)
        # create a mapping from characters to integers
        stoi = { ch:i for i,ch in enumerate(chars) }
        itos = { i:ch for i,ch in enumerate(chars) }
        self.encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
        self.decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
        self.data = torch.tensor(self.encode(self.data), dtype=torch.long)

    def __len__(self):
        return len(self.dataset_size)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [3]:
dataset = datasets.load_dataset("karpathy/tiny_shakespeare", split="train")

text_size = len(dataset[0]["text"])
data = dataset[0]["text"].replace("\n", "")
context_length = 1024
dataset_size = torch.arange(0, text_size-1, context_length)
x = [data[i:i+context_length] for i in dataset_size]

ds = datasets.Dataset.from_dict({"text": x})

tokenized_dataset = tokenize_and_concatenate(ds, rhyme_model.tokenizer, streaming=False, max_length=rhyme_model.cfg.n_ctx, column_name="text", add_bos_token=True, num_proc=4)

dataset_dict = tokenized_dataset.train_test_split(test_size=50)
train_loader = DataLoader(dataset_dict["train"], batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(dataset_dict["test"], batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

/cta/users/basan/.conda/envs/mech/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for karpathy/tiny_shakespeare contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/karpathy/tiny_shakespeare
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Map (num_proc=4):  25%|██▌       | 246/981 [00:00<00:00, 1155.29 examples/s]Token indices sequence length is longer than the specified maximum sequence length for this model (3648 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3646 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence le

In [4]:
@dataclass
class TransformerTrainingArgs():
	batch_size = 16
	epochs = num_epochs
	max_steps_per_epoch = 9
	lr = 1e-3
	weight_decay = 1e-3
	wandb_project: Optional[str] = "day1-demotransformer"
	wandb_name: Optional[str] = None

args = TransformerTrainingArgs()

def get_log_probs(
    logits: Float[Tensor, "batch posn d_vocab"],
    tokens: Int[Tensor, "batch posn"]
) -> Float[Tensor, "batch posn-1"]:

    log_probs = logits.log_softmax(dim=-1)
    # Get logprobs the first seq_len-1 predictions (so we can compare them with the actual next tokens)
    log_probs_for_tokens = log_probs[:, :-1].gather(dim=-1, index=tokens[:, 1:].unsqueeze(-1)).squeeze(-1)

    return log_probs_for_tokens


In [5]:
class TransformerTrainer:
    def __init__(self, args: TransformerTrainingArgs, model):
        super().__init__()
        self.model = model
        self.args = args
        self.optimizer = t.optim.AdamW(self.model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
        self.step = 0


    def training_step(self, batch: Dict[str, Int[Tensor, "batch seq"]]) -> Float[Tensor, ""]:
        '''
        Calculates the loss on the tokens in the batch, performs a gradient update step, and logs the loss.

        Remember that `batch` is a dictionary with the single key 'tokens'.
        '''
        # SOLUTION
        tokens = batch["tokens"].to(device)
        logits = self.model(tokens)
        loss = -get_log_probs(logits, tokens).mean()
        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()
        self.step += 1
        return loss


    def validation_step(self, batch: Dict[str, Int[Tensor, "batch seq"]]):
        '''
        Calculates & returns the accuracy on the tokens in the batch (i.e. how often the model's prediction
        is correct). Logging should happen in the `train` function (after we've computed the accuracy for
        the whole validation set).
        '''
        # SOLUTION
        tokens = batch["tokens"].to(device)
        logits: Tensor = self.model(tokens)[:, :-1]
        predicted_tokens = logits.argmax(dim=-1)
        correct_predictions = (predicted_tokens == tokens[:, 1:]).flatten()
        return correct_predictions


    def train(self):
        '''
        Trains the model, for `self.args.epochs` epochs. Also handles wandb initialisation, and early stopping
        for each epoch at `self.args.max_steps_per_epoch` steps.
        '''
        # SOLUTION
        accuracy = np.nan

        progress_bar = tqdm(total = self.args.max_steps_per_epoch * self.args.epochs)

        for epoch in range(self.args.epochs):
            for i, batch in enumerate(self.train_loader()):
                loss = self.training_step(batch)
                progress_bar.update()
                progress_bar.set_description(f"Epoch {epoch+1}, loss: {loss:.3f}, accuracy: {accuracy:.2f}")
                if i >= self.args.max_steps_per_epoch:
                    break

            correct_predictions = t.concat([self.validation_step(batch) for batch in self.test_loader()])
            accuracy = correct_predictions.float().mean().item()

    def train_loader(self) -> DataLoader:
        '''Returns train loader (as in code above).'''
        return DataLoader(dataset_dict["train"], batch_size=self.args.batch_size, shuffle=True, num_workers=4, pin_memory=True)


    def test_loader(self) -> DataLoader:
        '''Returns test loader (as in code above).'''
        return DataLoader(dataset_dict["test"], batch_size=self.args.batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [6]:
model = rhyme_model.cuda()

optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=betas, weight_decay=wd)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda i: min(i/100, 1.))

Moving model to device:  cuda


In [3]:
def load_model_state(model: HookedTransformer, filename: str) -> None:
    assert os.path.isdir("models"), "Make a directory `models` with model state dicts"
    if not filename.startswith("models/"):
        filename = f"models/{filename}"
    with open(filename, "rb") as f:
        state_dict = pickle.load(f)
    model.load_state_dict(state_dict)

In [8]:
import pickle
from datetime import datetime as dt

def save_model_state_dict(
    model: HookedTransformer, 
    filename: str | None = None
) -> None:
    if not os.path.isdir("models"):
        os.mkdir("models")
    if not filename:
        timestamp = dt.now().isoformat("T", "minutes").replace(":", "-")
        filename = f"model_state_dict_{timestamp}.pkl"
    with open(f"models/{filename}", "wb") as f:
        pickle.dump(model.state_dict(), f)

save_model_state_dict(model, "eminem2.pickle")

In [9]:
args = TransformerTrainingArgs()
trainer = TransformerTrainer(args, model)
trainer.train()

Epoch 150, loss: 2.065, accuracy: 0.17: : 300it [06:46,  1.36s/it]                      


# Attention maps for Rhyme GPT2 and Normal GPT2

Interpreting with Activation Pattern tool to see if I can spot an investigation point. Attention heads of GPT2 are not activated on the rhyming words before fine-tuning. In the fine-tuned model, attention heads that activate on rhyming words start to appear. Some neurons seem to gain the ability to attend to the previous rhyme.


### Attention Maps for Fine-tuned Rhyme GPT2 Small Model on "Rhymes"

In [7]:
text = """I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
I got a laptop in my back pocket
My pen'll go off when I half-cock it
"""

text = """Hush, little baby, don't say a word
Papa's gonna buy you a mockingbird."""

rhyme_model = HookedTransformer.from_pretrained("gpt2-small")
load_model_state(rhyme_model, "models/eminem.pickle")
model_logits, rhyme_cache = rhyme_model.run_with_cache(text, remove_batch_dim=True)
attention_pattern = rhyme_cache["pattern", 0]
print(attention_pattern.shape)
gpt2_str_tokens = rhyme_model.to_str_tokens(text)

for layer in range(rhyme_model.cfg.n_layers):
    display(cv.attention.attention_patterns(
        tokens=gpt2_str_tokens,
        attention=rhyme_cache["pattern", layer],
    ))

Loaded pretrained model gpt2-small into HookedTransformer
torch.Size([12, 23, 23])


### Attention Maps for GPT2 Small Model on "Rhymes"

In [5]:
text = """I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
I got a laptop in my back pocket
My pen'll go off when I half-cock it
"""

text = """Hush, little baby, don't say a word
Papa's gonna buy you a mockingbird."""

model = HookedTransformer.from_pretrained("gpt2-small")
model_logits, model_cache = model.run_with_cache(text, remove_batch_dim=True)
attention_pattern = model_cache["pattern", 0]
gpt2_str_tokens = model.to_str_tokens(text)

for layer in range(model.cfg.n_layers):
    display(cv.attention.attention_patterns(
        tokens=gpt2_str_tokens,
        attention=model_cache["pattern", layer],
    ))


Loaded pretrained model gpt2-small into HookedTransformer


## Test if Rhyme Model tends to predict more rhymed words

Hypothesis: If a model would like to produce rhymes, it should increase the possibility that a rhyme word will be predicted, even though we lose some meaning from the sentence. (The training dataset has meaningful rhymes, but there can be a slight favor towards words that rhyme with some other word.) 

I conducted a logit difference experiment on some rhyme sentences. Model sometimes favors rhymes, but in the first sentence, it does not favor bird over cat. I am sharing some useful examples here. You can experiment with it at Colab. Another striking result is that “it” is favored against “them, those and em” which are the words that grammatically completes the sentence better.

In [18]:
gpt2_small = HookedTransformer.from_pretrained("gpt2-small")
torch.manual_seed(0)

prompts = [
    "Hush, little baby, don't say a word, Papa's gonna buy you a",
    "Hush, little baby, don't say a word, Papa's gonna buy you a",
    "Hush, little baby, don't say a word, Papa's gonna buy you a",
    "Humpty Dumpty sat on a wall, Humpty Dumpty had a great",
    "Humpty Dumpty sat on a wall, Humpty Dumpty had a great",
    "I got laptops in my back pocket, My pen'll go off when I half-cock",
    "I got laptops in my back pocket, My pen'll go off when I half-cock",
    "I got laptops in my back pocket, My pen'll go off when I half-cock"
]
answers = [
    (" bird", " milk"),
    (" bird", " toy"),
    (" bird", " cat"),
    (" fall", " nap"),
    (" fall", " juice"),
    (" it", " them"),
    (" it", " those"),
    (" it", " em"),
]
answer_tokens = []
for i in range(len(prompts)):
    answer_tokens.append(
        (
            rhyme_model.to_single_token(answers[i][0]),
            rhyme_model.to_single_token(answers[i][1]),
        )
    )
answer_tokens = torch.tensor(answer_tokens).to("cuda:0")
print(prompts)
print(answers)

tokens = gpt2_small.to_tokens(prompts, prepend_bos=True)

source_logits, source_cache = gpt2_small.run_with_cache(tokens)
rhyme_logits, rlhf_cache = rhyme_model.run_with_cache(tokens)

def logits_to_ave_logit_diff(logits, answer_tokens, per_prompt=False):
    # Only the final logits are relevant for the answer
    final_logits = logits[:, -1, :]
    answer_logits = final_logits.gather(dim=-1, index=answer_tokens)
    answer_logit_diff = answer_logits[:, 0] - answer_logits[:, 1]
    if per_prompt:
        return answer_logit_diff
    else:
        return answer_logit_diff.mean()

print("RHYME GPT2")

print(
    "Per prompt logit difference:",
    logits_to_ave_logit_diff(rhyme_logits, answer_tokens, per_prompt=True)
    .detach()
    .cpu()
    .round(decimals=3),
)
original_average_logit_diff = logits_to_ave_logit_diff(rhyme_logits, answer_tokens)
print(
    "Average logit difference:",
    round(logits_to_ave_logit_diff(rhyme_logits, answer_tokens).item(), 3),
)

print("GPT2")

print(
    "Per prompt logit difference:",
    logits_to_ave_logit_diff(source_logits, answer_tokens, per_prompt=True)
    .detach()
    .cpu()
    .round(decimals=3),
)
original_average_logit_diff = logits_to_ave_logit_diff(source_logits, answer_tokens)
print(
    "Average logit difference:",
    round(logits_to_ave_logit_diff(source_logits, answer_tokens).item(), 3),
)

Loaded pretrained model gpt2-small into HookedTransformer
["Hush, little baby, don't say a word, Papa's gonna buy you a", "Hush, little baby, don't say a word, Papa's gonna buy you a", "Hush, little baby, don't say a word, Papa's gonna buy you a", 'Humpty Dumpty sat on a wall, Humpty Dumpty had a great', 'Humpty Dumpty sat on a wall, Humpty Dumpty had a great', "I got laptops in my back pocket, My pen'll go off when I half-cock", "I got laptops in my back pocket, My pen'll go off when I half-cock", "I got laptops in my back pocket, My pen'll go off when I half-cock"]
[(' bird', ' milk'), (' bird', ' toy'), (' bird', ' cat'), (' fall', ' nap'), (' fall', ' juice'), (' it', ' them'), (' it', ' those'), (' it', ' em')]
RHYME GPT2
Per prompt logit difference: tensor([ 4.2740, 10.6380, -5.1420,  7.3000,  3.5810,  4.5020,  5.2410, 12.1840])
Average logit difference: 5.322
GPT2
Per prompt logit difference: tensor([-2.2960, -2.5400, -1.0430,  0.5670,  1.5050,  1.5950,  6.3450,  8.8240])
Averag

## Is there Rhyme Heads? Detection of a rhyme head on a tiny dataset

In [8]:
rhymes_dataset = {
"lyrics":[

"""Your belly's answer? What!
The kingly-crowned head, the vigilant eye,
The counsellor heart, the arm our soldier,
Our steed the leg, the tongue our trumpeter.
With other muniments and petty helps
In this our fabric, if that they
""",
"""Hush, little baby, don't say a word
Papa's gonna buy you a mockingbird.""",

"""Humpty Dumpty sat on a wall,
Humpty Dumpty had a great fall.
All the King's horses and all the King's men,
Couldn't put Humpty together again.""",

"""I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
I got a laptop in my back pocket
My pen'll go off when I half-cock it
""",

"""Twinkle, twinkle, little star,
How I wonder what you are!""",

"""Then you show your little light,
Twinkle, twinkle, all the night.
""",

"""In the dark blue sky you keep,
And often through my curtains peep,
""",

"""As your bright and tiny spark,
Lights the traveller in the dark
""",

"""Romance, who loves to nod and sing,
With drowsy head and folded wing,
Among the green leaves as they shake
Far down within some shadowy lake"""
],
"positions":{}
}

rhymes = [[(" soldier", "eter")], [(" word", "bird")], [(" wall", " fall")], [(" pocket", " it")], [(" star", " are")],  [(" light", " night")], [(" keep", "ep")], [(" spark", " dark")],  [(" wing", " sing"), (" shake", " lake")]]
lyrics = rhymes_dataset["lyrics"]

for i, lyric in enumerate(lyrics):
    positions_per_lyric = []
    tokens = rhyme_model.to_str_tokens(lyric)
    print(tokens)
    for rhym in rhymes[i]:
        pos_1 = tokens.index(rhym[0])
        pos_2 = tokens.index(rhym[1])
        positions_per_lyric.append((pos_1, pos_2))

    rhymes_dataset["positions"].update({i:positions_per_lyric})
        
def rhyme_head_detector(cache: ActivationCache) -> List[str]:
    '''
    Returns a list e.g. ["0.2", "1.4", "1.9"] of "layer.head" which you judge to be induction heads

    Remember - the tokens used to generate rep_cache are (bos_token, *rand_tokens, *rand_tokens)
    '''
    layers = []


    for layer in range(rhyme_model.cfg.n_layers):
        for i, head in enumerate(range(12)):
            scores = []
            for j, lyric in enumerate(rhymes_dataset["lyrics"]):
                str_tokens = rhyme_model.to_str_tokens(lyric)
                tokens = rhyme_model.to_tokens(lyric)
                _, rhyme_cache = rhyme_model.run_with_cache(tokens, remove_batch_dim=True)
                attention_pattern = rhyme_cache["pattern", layer][head]
                indexes = rhymes_dataset["positions"][j]
                for idx in indexes:
                    attn_score = attention_pattern[idx[1], idx[0]]
                    scores.append(attn_score.cpu())
            
            if np.mean(scores) > 0.3:
                print(scores)
                layers.append(str(layer)+"."+str(i))

    return layers

layers = rhyme_head_detector(rhyme_model)
print("Rhyme Head Layers: ", layers)

['<|endoftext|>', 'Your', ' belly', "'s", ' answer', '?', ' What', '!', '\n', 'The', ' k', 'ingly', '-', 'c', 'rown', 'ed', ' head', ',', ' the', ' vigilant', ' eye', ',', '\n', 'The', ' coun', 'sell', 'or', ' heart', ',', ' the', ' arm', ' our', ' soldier', ',', '\n', 'Our', ' st', 'eed', ' the', ' leg', ',', ' the', ' tongue', ' our', ' trump', 'eter', '.', '\n', 'With', ' other', ' mun', 'iments', ' and', ' petty', ' helps', '\n', 'In', ' this', ' our', ' fabric', ',', ' if', ' that', ' they', '\n']
['<|endoftext|>', 'H', 'ush', ',', ' little', ' baby', ',', ' don', "'t", ' say', ' a', ' word', '\n', 'P', 'apa', "'s", ' gonna', ' buy', ' you', ' a', ' mocking', 'bird', '.']
['<|endoftext|>', 'Hum', 'pty', ' Dum', 'pty', ' sat', ' on', ' a', ' wall', ',', '\n', 'Hum', 'pty', ' Dum', 'pty', ' had', ' a', ' great', ' fall', '.', '\n', 'All', ' the', ' King', "'s", ' horses', ' and', ' all', ' the', ' King', "'s", ' men', ',', '\n', 'Could', 'n', "'t", ' put', ' Hum', 'pty', ' together'

## Check if Indirect Object Identification is preserved

Model is not able to complete classical IOI examples after fine-tuning with the same performance. Classical example used in IOI. 

In [9]:
example_prompt = "After John and Mary went to the store, John gave a bottle of milk to"
example_answer = " Mary"
print("RHYME MODEL")
utils.test_prompt(example_prompt, example_answer, rhyme_model, prepend_bos=True)

gpt2_small = HookedTransformer.from_pretrained("gpt2-small")
example_prompt = "After John and Mary went to the store, John gave a bottle of milk to"
example_answer = " Mary"
print("GPT2 SMALL")
utils.test_prompt(example_prompt, example_answer, gpt2_small, prepend_bos=True)

RHYME MODEL
Tokenized prompt: ['<|endoftext|>', 'After', ' John', ' and', ' Mary', ' went', ' to', ' the', ' store', ',', ' John', ' gave', ' a', ' bottle', ' of', ' milk', ' to']
Tokenized answer: [' Mary']


Performance on answer token:
Rank: 60       Logit: 15.31 Prob:  0.00% Token: | Mary|

Top 0th token. Logit: 28.35 Prob: 99.40% Token: | the|
Top 1th token. Logit: 22.89 Prob:  0.42% Token: | his|
Top 2th token. Logit: 20.96 Prob:  0.06% Token: | one|
Top 3th token. Logit: 20.25 Prob:  0.03% Token: | England|
Top 4th token. Logit: 19.60 Prob:  0.02% Token: | Port|
Top 5th token. Logit: 18.68 Prob:  0.01% Token: | him|
Top 6th token. Logit: 18.45 Prob:  0.00% Token: | Bristol|
Top 7th token. Logit: 18.10 Prob:  0.00% Token: | nine|
Top 8th token. Logit: 17.83 Prob:  0.00% Token: | Hy|
Top 9th token. Logit: 17.64 Prob:  0.00% Token: | Barb|


Ranks of the answer tokens: [(' Mary', 60)]

Loaded pretrained model gpt2-small into HookedTransformer
GPT2 SMALL
Tokenized prompt: ['<|endoftext|>', 'After', ' John', ' and', ' Mary', ' went', ' to', ' the', ' store', ',', ' John', ' gave', ' a', ' bottle', ' of', ' milk', ' to']
Tokenized answer: [' Mary']


Performance on answer token:
Rank: 0        Logit: 18.09 Prob: 70.07% Token: | Mary|

Top 0th token. Logit: 18.09 Prob: 70.07% Token: | Mary|
Top 1th token. Logit: 15.38 Prob:  4.67% Token: | the|
Top 2th token. Logit: 15.35 Prob:  4.54% Token: | John|
Top 3th token. Logit: 15.25 Prob:  4.11% Token: | them|
Top 4th token. Logit: 14.84 Prob:  2.73% Token: | his|
Top 5th token. Logit: 14.06 Prob:  1.24% Token: | her|
Top 6th token. Logit: 13.54 Prob:  0.74% Token: | a|
Top 7th token. Logit: 13.52 Prob:  0.73% Token: | their|
Top 8th token. Logit: 13.13 Prob:  0.49% Token: | Jesus|
Top 9th token. Logit: 12.97 Prob:  0.42% Token: | him|


Ranks of the answer tokens: [(' Mary', 0)]

## Indirect Object Identification next logit checking

In [10]:
prompt_format = [
    "When John and Mary went to the shops,{} gave the bag to",
    "When Tom and James went to the park,{} gave the ball to",
    "When Dan and Sid went to the shops,{} gave an apple to",
    "After Martin and Amy went to the park,{} gave a drink to",
]
names = [
    (" Mary", " John"),
    (" Tom", " James"),
    (" Dan", " Sid"),
    (" Martin", " Amy"),
]
# List of prompts
prompts = []
# List of answers, in the format (correct, incorrect)
answers = []
# List of the token (ie an integer) corresponding to each answer, in the format (correct_token, incorrect_token)
answer_tokens = []
for i in range(len(prompt_format)):
    for j in range(2):
        answers.append((names[i][j], names[i][1 - j]))
        answer_tokens.append(
            (
                gpt2_small.to_single_token(answers[-1][0]),
                gpt2_small.to_single_token(answers[-1][1]),
            )
        )
        # Insert the *incorrect* answer to the prompt, making the correct answer the indirect object.
        prompts.append(prompt_format[i].format(answers[-1][1]))
answer_tokens = torch.tensor(answer_tokens).to(device)
print(prompts)
print(answers)

tokens = gpt2_small.to_tokens(prompts, prepend_bos=True)

source_logits, source_cache = gpt2_small.run_with_cache(tokens)
rhyme_logits, rlhf_cache = rhyme_model.run_with_cache(tokens)

def logits_to_ave_logit_diff(logits, answer_tokens, per_prompt=False):
    # Only the final logits are relevant for the answer
    final_logits = logits[:, -1, :]
    answer_logits = final_logits.gather(dim=-1, index=answer_tokens)
    answer_logit_diff = answer_logits[:, 0] - answer_logits[:, 1]
    if per_prompt:
        return answer_logit_diff
    else:
        return answer_logit_diff.mean()

print("RHYME GPT2")

print(
    "Per prompt logit difference:",
    logits_to_ave_logit_diff(rhyme_logits, answer_tokens, per_prompt=True)
    .detach()
    .cpu()
    .round(decimals=3),
)
original_average_logit_diff = logits_to_ave_logit_diff(rhyme_logits, answer_tokens)
print(
    "Average logit difference:",
    round(logits_to_ave_logit_diff(rhyme_logits, answer_tokens).item(), 3),
)

print("GPT2")

print(
    "Per prompt logit difference:",
    logits_to_ave_logit_diff(source_logits, answer_tokens, per_prompt=True)
    .detach()
    .cpu()
    .round(decimals=3),
)
original_average_logit_diff = logits_to_ave_logit_diff(source_logits, answer_tokens)
print(
    "Average logit difference:",
    round(logits_to_ave_logit_diff(source_logits, answer_tokens).item(), 3),
)

['When John and Mary went to the shops, John gave the bag to', 'When John and Mary went to the shops, Mary gave the bag to', 'When Tom and James went to the park, James gave the ball to', 'When Tom and James went to the park, Tom gave the ball to', 'When Dan and Sid went to the shops, Sid gave an apple to', 'When Dan and Sid went to the shops, Dan gave an apple to', 'After Martin and Amy went to the park, Amy gave a drink to', 'After Martin and Amy went to the park, Martin gave a drink to']
[(' Mary', ' John'), (' John', ' Mary'), (' Tom', ' James'), (' James', ' Tom'), (' Dan', ' Sid'), (' Sid', ' Dan'), (' Martin', ' Amy'), (' Amy', ' Martin')]
RHYME GPT2
Per prompt logit difference: tensor([  8.3550,  -8.4730,  -2.5410,  -2.2200,  -0.8670, -10.5630,   8.6060,
        -13.1560])
Average logit difference: -2.608
GPT2
Per prompt logit difference: tensor([3.3370, 3.2020, 2.7090, 3.7970, 1.7200, 5.2810, 2.6010, 5.7670])
Average logit difference: 3.552


## Plotting code from plotly_utils from ARENA

### Check Induction Heads in both models

In [ ]:
update_layout_set = {"xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat", "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth", "yaxis_gridcolor", "showlegend", "xaxis_tickmode", "yaxis_tickmode", "margin", "xaxis_visible", "yaxis_visible", "bargap", "bargroupgap", "coloraxis_showscale", "xaxis_tickangle", "yaxis_scaleanchor", "xaxis_tickfont", "yaxis_tickfont"}
update_traces_set = {"textposition"}

def to_numpy(tensor):
    """
    Helper function to convert a tensor to a numpy array. Also works on lists, tuples, and numpy arrays.
    """
    if isinstance(tensor, np.ndarray):
        return tensor
    elif isinstance(tensor, (list, tuple)):
        array = np.array(tensor)
        return array
    elif isinstance(tensor, (t.Tensor, t.nn.parameter.Parameter)):
        return tensor.detach().cpu().numpy()
    elif isinstance(tensor, (int, float, bool, str)):
        return np.array(tensor)
    else:
        raise ValueError(f"Input to to_numpy has invalid type: {type(tensor)}")

def reorder_list_in_plotly_way(L: list, col_wrap: int):
    '''
    Helper function, because Plotly orders figures in an annoying way when there's column wrap.
    '''
    L_new = []
    while len(L) > 0:
        L_new.extend(L[-col_wrap:])
        L = L[:-col_wrap]
    return L_new

def imshow(tensor: t.Tensor, renderer=None, **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    if ("size" in kwargs_pre) or ("shape" in kwargs_pre):
        size = kwargs_pre.pop("size", None) or kwargs_pre.pop("shape", None)
        kwargs_pre["height"], kwargs_pre["width"] = size
    facet_labels = kwargs_pre.pop("facet_labels", None)
    border = kwargs_pre.pop("border", False)
    return_fig = kwargs_pre.pop("return_fig", False)
    text = kwargs_pre.pop("text", None)
    xaxis_tickangle = kwargs_post.pop("xaxis_tickangle", None)
    # xaxis_tickfont = kwargs_post.pop("xaxis_tickangle", None)
    static = kwargs_pre.pop("static", False)
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    if "color_continuous_midpoint" not in kwargs_pre:
        kwargs_pre["color_continuous_midpoint"] = 0.0
    if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
        kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
    fig = px.imshow(to_numpy(tensor), **kwargs_pre).update_layout(**kwargs_post)
    if facet_labels:
        # Weird thing where facet col wrap means labels are in wrong order
        if "facet_col_wrap" in kwargs_pre:
            facet_labels = reorder_list_in_plotly_way(facet_labels, kwargs_pre["facet_col_wrap"])
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label
    if border:
        fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
        fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    if text:
        if tensor.ndim == 2:
            # if 2D, then we assume text is a list of lists of strings
            assert isinstance(text[0], list)
            assert isinstance(text[0][0], str)
            text = [text]
        else:
            # if 3D, then text is either repeated for each facet, or different
            assert isinstance(text[0], list)
            if isinstance(text[0][0], str):
                text = [text for _ in range(len(fig.data))]
        for i, _text in enumerate(text):
            fig.data[i].update(
                text=_text, 
                texttemplate="%{text}", 
                textfont={"size": 12}
            )
    # Very hacky way of fixing the fact that updating layout with xaxis_* only applies to first facet by default
    if xaxis_tickangle is not None:
        n_facets = 1 if tensor.ndim == 2 else tensor.shape[0]
        for i in range(1, 1+n_facets):
            xaxis_name = "xaxis" if i == 1 else f"xaxis{i}"
            fig.layout[xaxis_name]["tickangle"] = xaxis_tickangle
    return fig if return_fig else fig.show(renderer=renderer, config={"staticPlot": static})


In [ ]:
def generate_repeated_tokens(
    model: HookedTransformer, seq_len: int, batch: int = 1
) -> Int[Tensor, "batch full_seq_len"]:
    '''
    Generates a sequence of repeated random tokens

    Outputs are:
        rep_tokens: [batch, 1+2*seq_len]
    '''
    prefix = (t.ones(batch, 1) * model.tokenizer.bos_token_id).long() # add the prefix (BOS)
    rep_tokens_half = t.randint(0, model.cfg.d_vocab, (batch, seq_len), dtype=t.int64)
    rep_tokens = t.cat([prefix, rep_tokens_half, rep_tokens_half], dim=-1).to(device)
    return rep_tokens

def run_and_cache_model_repeated_tokens(model: HookedTransformer, seq_len: int, batch: int = 1) -> Tuple[t.Tensor, t.Tensor, ActivationCache]:
    '''
    Generates a sequence of repeated random tokens, and runs the model on it, returning logits, tokens and cache

    Should use the `generate_repeated_tokens` function above

    Outputs are:
        rep_tokens: [batch, 1+2*seq_len]
        rep_logits: [batch, 1+2*seq_len, d_vocab]
        rep_cache: The cache of the model run on rep_tokens
    '''
    rep_tokens = generate_repeated_tokens(model, seq_len, batch)
    rep_logits, rep_cache = model.run_with_cache(rep_tokens)
    return rep_tokens, rep_logits, rep_cache

seq_len = 50
batch = 10
# (rep_tokens, rep_logits, rep_cache) = run_and_cache_model_repeated_tokens(gpt2_small, seq_len, batch)
# rep_str = gpt2_small.to_str_tokens(rep_tokens)

rep_tokens_10 = generate_repeated_tokens(gpt2_small, seq_len, batch)

# We make a tensor to store the induction score for each head.
# We put it on the model's device to avoid needing to move things between the GPU and CPU, which can be slow.
induction_score_store = t.zeros((gpt2_small.cfg.n_layers, gpt2_small.cfg.n_heads), device=gpt2_small.cfg.device)

def induction_score_hook(
    pattern: Float[Tensor, "batch head_index dest_pos source_pos"],
    hook: HookPoint,
):
    
    '''
    Calculates the induction score, and stores it in the [layer, head] position of the `induction_score_store` tensor.
    '''
    indexing = seq_len-1
    induction_score = pattern.diagonal(offset=-indexing, dim1=-2, dim2=-1)
    # avg score per head
    induction_score = einops.rearrange(induction_score, 'batch head_idx layer -> batch layer head_idx')
    induction_score_store[hook.layer(), :] = induction_score.mean(dim=(0,1))

pattern_hook_names_filter = lambda name: name.endswith("pattern")

# Run with hooks (this is where we write to the `induction_score_store` tensor`)
gpt2_small.run_with_hooks(
    rep_tokens_10,
    return_type=None, # For efficiency, we don't need to calculate the logits
    fwd_hooks=[(
        pattern_hook_names_filter,
        induction_score_hook
    )]
)

# Plot the induction scores for each head in each layer
imshow(
    induction_score_store,
    labels={"x": "Head", "y": "Layer"},
    title="Induction Score by Head",
    text_auto=".2f",
    width=900, height=400
)

In [ ]:
update_layout_set = {"xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat", "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth", "yaxis_gridcolor", "showlegend", "xaxis_tickmode", "yaxis_tickmode", "margin", "xaxis_visible", "yaxis_visible", "bargap", "bargroupgap", "coloraxis_showscale", "xaxis_tickangle", "yaxis_scaleanchor", "xaxis_tickfont", "yaxis_tickfont"}

update_traces_set = {"textposition"}


def generate_repeated_tokens(
    model: HookedTransformer, seq_len: int, batch: int = 1
) -> Int[Tensor, "batch full_seq_len"]:
    '''
    Generates a sequence of repeated random tokens

    Outputs are:
        rep_tokens: [batch, 1+2*seq_len]
    '''
    prefix = (t.ones(batch, 1) * model.tokenizer.bos_token_id).long() # add the prefix (BOS)
    rep_tokens_half = t.randint(0, model.cfg.d_vocab, (batch, seq_len), dtype=t.int64)
    rep_tokens = t.cat([prefix, rep_tokens_half, rep_tokens_half], dim=-1).to(device)
    return rep_tokens

def run_and_cache_model_repeated_tokens(model: HookedTransformer, seq_len: int, batch: int = 1) -> Tuple[t.Tensor, t.Tensor, ActivationCache]:
    '''
    Generates a sequence of repeated random tokens, and runs the model on it, returning logits, tokens and cache

    Should use the `generate_repeated_tokens` function above

    Outputs are:
        rep_tokens: [batch, 1+2*seq_len]
        rep_logits: [batch, 1+2*seq_len, d_vocab]
        rep_cache: The cache of the model run on rep_tokens
    '''
    rep_tokens = generate_repeated_tokens(model, seq_len, batch)
    rep_logits, rep_cache = model.run_with_cache(rep_tokens)
    return rep_tokens, rep_logits, rep_cache

seq_len = 50
batch = 10
# (rep_tokens, rep_logits, rep_cache) = run_and_cache_model_repeated_tokens(gpt2_small, seq_len, batch)
# rep_str = gpt2_small.to_str_tokens(rep_tokens)

rep_tokens_10 = generate_repeated_tokens(rhyme_model, seq_len, batch)

# We make a tensor to store the induction score for each head.
# We put it on the model's device to avoid needing to move things between the GPU and CPU, which can be slow.
induction_score_store = t.zeros((rhyme_model.cfg.n_layers, rhyme_model.cfg.n_heads), device=rhyme_model.cfg.device)

def induction_score_hook(
    pattern: Float[Tensor, "batch head_index dest_pos source_pos"],
    hook: HookPoint,
):
    
    '''
    Calculates the induction score, and stores it in the [layer, head] position of the `induction_score_store` tensor.
    '''
    indexing = seq_len-1
    induction_score = pattern.diagonal(offset=-indexing, dim1=-2, dim2=-1)
    # avg score per head
    induction_score = einops.rearrange(induction_score, 'batch head_idx layer -> batch layer head_idx')
    induction_score_store[hook.layer(), :] = induction_score.mean(dim=(0,1))

pattern_hook_names_filter = lambda name: name.endswith("pattern")

# Run with hooks (this is where we write to the `induction_score_store` tensor`)
rhyme_model.run_with_hooks(
    rep_tokens_10,
    return_type=None, # For efficiency, we don't need to calculate the logits
    fwd_hooks=[(
        pattern_hook_names_filter,
        induction_score_hook
    )]
)

# Plot the induction scores for each head in each layer
imshow(
    induction_score_store,
    labels={"x": "Head", "y": "Layer"},
    title="Induction Score by Head",
    text_auto=".2f",
    width=900, height=400
)